In [ ]:
pip install torch numpy


2. Time2Vec モジュールの実装
Time2Vecは、時系列データに時間情報を加えるための方法です。以下にその実装を示します。

In [1]:
import torch
import torch.nn as nn


class Time2Vec(nn.Module):
    def __init__(self, kernel_size):
        super(Time2Vec, self).__init__()
        self.k = kernel_size
        self.w = nn.Parameter(torch.randn(kernel_size))
        self.b = nn.Parameter(torch.randn(kernel_size))
        self.w0 = nn.Parameter(torch.randn(1))
        self.b0 = nn.Parameter(torch.randn(1))

    def forward(self, x):
        v1 = self.w0 * x + self.b0
        v2 = torch.sin(self.w * x + self.b)
        return torch.cat([v1, v2], dim=-1)



3. Transformer Encoder with LSTM
次に、LSTMをフィードフォーワード層として使用するTransformer Encoderを実装します。



In [ ]:
class TransformerEncoderLayerWithLSTM(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward, dropout=0.1):
        super(TransformerEncoderLayerWithLSTM, self).__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout)
        self.lstm = nn.LSTM(d_model, dim_feedforward, batch_first=True)
        self.linear1 = nn.Linear(dim_feedforward, dim_feedforward)
        self.dropout = nn.Dropout(dropout)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(dim_feedforward)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src):
        src2 = self.self_attn(src, src, src)[0]
        src = src + self.dropout1(src2)
        src = self.norm1(src)

        src2, _ = self.lstm(src.permute(1, 0, 2))  # Ensure batch_first=True
        src2 = self.linear1(src2.permute(1, 0, 2))
        src = src + self.dropout2(src2)
        src = self.norm2(src)
        return src


4. 全体のTransformerモデルの実装
最後に、Time2VecとTransformer Encoderを組み合わせてモデル全体を実装します。

In [ ]:
class TimeSeriesTransformer(nn.Module):
    def __init__(self, d_model, nhead, num_encoder_layers, dim_feedforward, time2vec_dim, dropout=0.1):
        super(TimeSeriesTransformer, self).__init__()
        self.time2vec = Time2Vec(time2vec_dim)
        self.embedding = nn.Linear(time2vec_dim + 2, d_model)  # Adjusted input dimension
        self.encoder_layers = nn.ModuleList([
            TransformerEncoderLayerWithLSTM(d_model, nhead, dim_feedforward, dropout)
            for _ in range(num_encoder_layers)
        ])
        self.linear = nn.Linear(d_model, 1)

    def forward(self, src):
        time_embed = self.time2vec(src)
        src = torch.cat([src, time_embed], dim=-1)
        src = self.embedding(src)
        src = src.permute(1, 0, 2)  # Adjust for MultiheadAttention shape requirement
        for layer in self.encoder_layers:
            src = layer(src)
        src = src.permute(1, 0, 2)  # Revert shape adjustment
        output = self.linear(src)
        return output


# ハイパーパラメータの設定
d_model = 64
nhead = 8
num_encoder_layers = 6
dim_feedforward = 64
time2vec_dim = 10
dropout = 0.1
#d_modelとdim_feedforwardを一致させないと動かないが，これは正しい設計なのか？？？

# モデルの初期化
model = TimeSeriesTransformer(d_model, nhead, num_encoder_layers, dim_feedforward, time2vec_dim, dropout)


5. モデルのトレーニング
トレーニングループを実装します。

In [ ]:
import torch
from sklearn.model_selection import train_test_split
import torch
import numpy as np

# ダミーデータの生成
batch_size=32
num_samples = 1000
sequence_length = 100

x = torch.randn(num_samples, sequence_length, 1)
y = torch.randn(num_samples, sequence_length, 1)

# データを時間順に訓練、検証、テストに分割
train_size = int(0.7 * num_samples)
val_size = int(0.15 * num_samples)
test_size = num_samples - train_size - val_size

x_train = x[:train_size]
y_train = y[:train_size]
x_val = x[train_size:train_size + val_size]
y_val = y[train_size:train_size + val_size]
x_test = x[train_size + val_size:]
y_test = y[train_size + val_size:]

# DataLoaderの作成（シャッフルなし）#時系列分析なのでshuffleしなかったけど正しいのか？
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
val_dataset = torch.utils.data.TensorDataset(x_val, y_val)
test_dataset = torch.utils.data.TensorDataset(x_test, y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



In [ ]:
import torch.optim as optim
import copy
import matplotlib.pyplot as plt


# 損失関数とオプティマイザの設定
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# トレーニングと検証ループ
num_epochs = 100
best_val_loss = float('inf')
best_model_wts = copy.deepcopy(model.state_dict())

for epoch in range(num_epochs):
    # トレーニングモード
    model.train()
    train_loss = 0.0
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(x_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * x_batch.size(0)

    train_loss /= len(train_loader.dataset)

    # 検証モード
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x_batch, y_batch in val_loader:
            output = model(x_batch)
            loss = criterion(output, y_batch)
            val_loss += loss.item() * x_batch.size(0)

    val_loss /= len(val_loader.dataset)

    # ベストモデルの保存
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model_wts = copy.deepcopy(model.state_dict())

    if (epoch + 1) % 5 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

# ベストモデルのロード
model.load_state_dict(best_model_wts)

# テストデータでの評価
model.eval()
test_loss = 0.0
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        output = model(x_batch)
        loss = criterion(output, y_batch)
        test_loss += loss.item() * x_batch.size(0)

test_loss /= len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}')


Epoch [5/100], Train Loss: 1.0081, Val Loss: 1.0110
Epoch [10/100], Train Loss: 1.0062, Val Loss: 1.0119
Epoch [15/100], Train Loss: 1.0040, Val Loss: 1.0128
Epoch [20/100], Train Loss: 1.0030, Val Loss: 1.0129
Epoch [25/100], Train Loss: 1.0029, Val Loss: 1.0132
Epoch [30/100], Train Loss: 1.0030, Val Loss: 1.0131
Epoch [35/100], Train Loss: 1.0024, Val Loss: 1.0135
Epoch [40/100], Train Loss: 1.0023, Val Loss: 1.0131
Epoch [45/100], Train Loss: 1.0021, Val Loss: 1.0135
Epoch [50/100], Train Loss: 1.0020, Val Loss: 1.0137
Epoch [55/100], Train Loss: 1.0011, Val Loss: 1.0137
Epoch [60/100], Train Loss: 1.0001, Val Loss: 1.0136
Epoch [65/100], Train Loss: 0.9987, Val Loss: 1.0150
Epoch [70/100], Train Loss: 0.9957, Val Loss: 1.0170
Epoch [75/100], Train Loss: 0.9915, Val Loss: 1.0200
Epoch [80/100], Train Loss: 0.9828, Val Loss: 1.0396
Epoch [85/100], Train Loss: 0.9721, Val Loss: 1.0408
Epoch [90/100], Train Loss: 0.9532, Val Loss: 1.0631
Epoch [95/100], Train Loss: 0.9240, Val Loss: 1

In [ ]:
import torch.optim as optim

# データの準備 (例: ダミーデータ)
batch_size = 32
sequence_length = 100
x = torch.randn(batch_size, sequence_length, 1)
y = torch.randn(batch_size, sequence_length, 1)

# 損失関数とオプティマイザの設定
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# トレーニングループ
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(x)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
